In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/billsum-processed-train/catest_processed.csv
/kaggle/input/billsum-processed-train/ustrain_processed.csv
/kaggle/input/billsum-processed-train/ustest_processed.csv
/kaggle/input/gemma/transformers/2b-it/2/model.safetensors.index.json
/kaggle/input/gemma/transformers/2b-it/2/gemma-2b-it.gguf
/kaggle/input/gemma/transformers/2b-it/2/config.json
/kaggle/input/gemma/transformers/2b-it/2/model-00001-of-00002.safetensors
/kaggle/input/gemma/transformers/2b-it/2/model-00002-of-00002.safetensors
/kaggle/input/gemma/transformers/2b-it/2/tokenizer.json
/kaggle/input/gemma/transformers/2b-it/2/tokenizer_config.json
/kaggle/input/gemma/transformers/2b-it/2/special_tokens_map.json
/kaggle/input/gemma/transformers/2b-it/2/.gitattributes
/kaggle/input/gemma/transformers/2b-it/2/tokenizer.model
/kaggle/input/gemma/transformers/2b-it/2/generation_config.json


In [3]:
# !pip install -U -q transformers langchain peft bitsandbytes trl datasets==2.16.0 notebook accelerate evaluate rouge

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.8.2 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cudf

In [4]:
from transformers import pipeline, set_seed
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from accelerate.utils import release_memory
import torch
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
from datasets import Dataset
from trl import SFTTrainer
from peft import LoraConfig, PeftModel
import pandas as pd
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter, HTMLHeaderTextSplitter
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
import evaluate
import transformers
from langchain.llms.base import LLM
from typing import Any
import warnings
import gc
import random
import numpy as np

warnings.filterwarnings('ignore')

# Set seed for reproducibility
set_seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Logging in HF
login(token = 'HF_TOKEN')

2024-03-29 23:53:30.046648: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 23:53:30.046762: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 23:53:30.173055: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
training = pd.read_csv('/kaggle/input/billsum-processed-train/ustrain_processed.csv')[['clean_text','summary']]
training.head()

,clean_text,summary
0,SECTIONHEADER SHORT TITLE. This Act may be cit...,Border Hospital Survival and Illegal Immigrant...
1,SECTIONHEADER SHORT TITLE. This Act may be cit...,Farm to School Improvements Act of 2010 - Amen...
2,SECTIONHEADER SHORT TITLE. This Act may be cit...,Persian Gulf War Illness Compensation Act of 2...
3,SECTIONHEADER SHORT TITLE. This Act may be cit...,Medicare Part D Outreach and Enrollment Enhanc...
4,SECTIONHEADER SHORT TITLE. This Act may be cit...,Seniors' Retirement Recovery Act of 2002 - Ame...


In [8]:
model = "/kaggle/input/gemma/transformers/2b-it/2"

lora_config = LoraConfig(
    r=6,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model)
tokenizer.padding_side = "right" # Fixing overflow issue ref: source code
model = AutoModelForCausalLM.from_pretrained(model, device_map="auto", quantization_config=bnb_config)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
training = training.head(10000)
train_data = Dataset.from_pandas(training)

test_df = pd.read_csv('/kaggle/input/billsum-processed-train/catest_processed.csv')[['clean_text','summary']]
testing_data = Dataset.from_pandas(test_df.head(10))

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['clean_text'])):
        messages = [
            {"role": "user",
             "content": "Given the following case, write a short summary of the case in 2-3 sentences:\n\nArticle: {}".format(example['clean_text'][i])},
            {"role": "assistant",
             "content": "{}".format(example['summary'][i])}
        ]
        output_texts.append(tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False))
        
    return output_texts

# Print the first training example
print(formatting_prompts_func(train_data[:1])[0])

<start_of_turn>user
Given the following case, write a short summary of the case in 2-3 sentences:

Article: SECTIONHEADER SHORT TITLE. This Act may be cited as the "Border Hospital Survival and Illegal Immigrant Care Act". SECTIONHEADER FINDINGS. The Congress finds as follows: Immigration is a Federal responsibility. The Immigration and Naturalization Service does not take into custody all aliens who are unlawfully present in the United States. Section 1867 of the Social Security Act and State laws require that, if any individual comes to a hospital and the hospital determines that the individual has an emergency medical condition, the hospital must provide either, within the staff and facilities available at the hospital, for such further medical examination and such treatment as may be required to stabilize the medical condition, or, if appropriate, for transfer of the individual to another medical facility. The Southwest border region is ill-equipped to absorb the expense of providi

In [10]:
# !pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=cdad5329f9fb2c3962b8422b6b9cf59321d5fd14c049c54fb686820f1514207c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [12]:
import numpy as np

import evaluate
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset = testing_data,
    max_seq_length=512,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=30,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        report_to='none',
        output_dir='logs',
        optim="paged_adamw_8bit"
    ),
    compute_metrics=compute_metrics,
    peft_config=lora_config,
    formatting_func=formatting_prompts_func,
)
trainer.train()

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Step,Training Loss
1,2.757300
2,3.018100
3,2.726800
4,2.767000
5,2.443500
6,2.471700
7,2.247600
8,1.987600
9,2.135600
10,2.041900


TrainOutput(global_step=30, training_loss=2.1362595240275066, metrics={'train_runtime': 138.791, 'train_samples_per_second': 0.865, 'train_steps_per_second': 0.216, 'total_flos': 733312985333760.0, 'train_loss': 2.1362595240275066, 'epoch': 0.01})

In [15]:
trainer.model.save_pretrained('lora_adapter')
# trainer.evaluate()

In [16]:
base_model_name = "/kaggle/input/gemma/transformers/2b-it/2"
adapter_model_name = "/kaggle/working/lora_adapter"

model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, adapter_model_name, device_map='auto', torch_dtype=torch.float16)

# Merge the adapters into the base model so you can use the model like a normal transformers model
model = model.merge_and_unload()
model.save_pretrained('final_model')

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
model = "/kaggle/working/final_model"

# Load the HF pipeline using our newly fine-tuned Gemma 2B
pipe_finetuned = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    model_kwargs={"torch_dtype": torch.float16},
    device_map='auto',
    max_new_tokens=512
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
text = '''SECTIONHEADER SHORT TITLE. This Act may be cited as the "National Science Education Tax Incentive for Businesses Act of 2007". SECTIONHEADER CREDITS FOR CERTAIN CONTRIBUTIONS BENEFITING SCIENCE, TECHNOLOGY, ENGINEERING, AND MATHEMATICS EDUCATION AT THE ELEMENTARY AND SECONDARY SCHOOL LEVEL. In General. Subpart D of part IV of subchapter A of chapter 1 of the Internal Revenue Code of 1986 is amended by adding at the end the following new section: "Section 45O. CONTRIBUTIONS BENEFITING SCIENCE, TECHNOLOGY, ENGINEERING, AND MATHEMATICS EDUCATION AT THE ELEMENTARY AND SECONDARY SCHOOL LEVEL. In General. For purposes of section 38, the elementary and secondary science, technology, engineering, and mathematics (STEM) contributions credit determined under this section for the taxable year is an amount equal to 100 percent of the qualified STEM contributions of the taxpayer for such taxable year. Qualified STEM Contributions. For purposes of this section, the term `qualified STEM contributions' means STEM school contributions, STEM teacher externship expenses, and STEM teacher training expenses. STEM School Contributions. For purposes of this section In general. The term `STEM school contributions' means STEM property contributions, and STEM service contributions. STEM property contributions. The term `STEM property contributions' means the amount which would (but for subsection ) be allowed as a deduction under section 170 for a charitable contribution of STEM inventory property if the donee is an elementary or secondary school described in section 170(b)(1)(A)(ii), substantially all of the use of the property by the donee is within the United States or within the defense dependents' education system for educational purposes in any of the grades K-12 that are related to the purpose or function of the donee, the original use of the property begins with the donee, the property will fit productively into the donee's education plan, the property is not transferred by the donee in exchange for money, other property, or services, except for shipping, installation and transfer costs, and the donee's use and disposition of the property will be in accordance with the provisions of subparagraphs (B) and (E). The determination of the amount of deduction under section 170 for purposes of this paragraph shall be made as if the limitation under section 170(e)(3)(B) applied to all STEM inventory property. STEM service contributions. The term `STEM service contributions' means the amount paid or incurred during the taxable year for STEM services provided in the United States or in the defense dependents' education system for the exclusive benefit of students at an elementary or secondary school described in section 170(b)(1)(A)(ii) but only if the taxpayer is engaged in the trade or business of providing such services on a commercial basis, and no charge is imposed for providing such services. STEM inventory property. The term `STEM inventory property' means, with respect to any contribution to a school, any property which is described in paragraph (1) or (2) of section 1221(a) with respect to the donor, and which is determined by the school to be needed by the school in providing education in grades K-12 in the areas of science, technology, engineering, or mathematics. STEM services. The term `STEM services' means, with respect to any contribution to a school, any service determined by the school to be needed by the school in providing education in grades K-12 in the areas of science, technology, engineering, or mathematics, including teaching courses of instruction at such school in any such area. Defense dependents' education system. For purposes of this subsection, the term `defense dependents' education system' means the program established and operated under the Defense Dependents' Education Act of 1978 . STEM Teacher Externship Expenses. For purposes of this section In general. The term `STEM teacher externship expenses' means any amount paid or incurred to carry out a STEM externship program of the taxpayer but only to the extent that such amount is attributable to the participation in such program of any eligible STEM teacher, including amounts paid to such a teacher as a stipend while participating in such program. STEM externship program. The term `STEM externship program' means any program established by a taxpayer engaged in a trade or business within an area of science, technology, engineering, or mathematics, and under which eligible STEM teachers receive training to enhance their teaching skills in the areas of science, technology, engineering, or mathematics or otherwise improve their knowledge in such areas. Eligible stem teacher. The term `eligible STEM teacher' means any individual who is a teacher in grades K-12 at an educational organization described in section 170(b)(1)(A)(ii) which is located in the United States or which is located on a United States military base outside the United States, and whose teaching responsibilities at such school include, or are likely to include, any course in the areas of science, technology, engineering, or mathematics. STEM Teacher Training Expenses. The term `STEM teacher training expenses' means any amount paid or incurred by a taxpayer engaged in a trade or business within an area of science, technology, engineering, or mathematics which is attributable to the participation of any eligible STEM teacher in a regular training program provided to employees of the taxpayer which is determined by such teacher's school as enhancing such teacher's teaching skills in the areas of science, technology, engineering, or mathematics. Denial of Double Benefit. No deduction shall be allowed under this chapter for any amount allowed as a credit under this section.". Conforming Amendments. Section 38(b) of such Code is amended by striking "plus" at the end of paragraph (30), by striking the period at the end of paragraph (31), and inserting ", plus", and by adding at the end the following new paragraph: the elementary and secondary science, technology, engineering, and mathematics (STEM) contributions credit determined under section 45O.". The table of sections for subpart D of part IV of subchapter A of chapter 1 of such Code is amended by adding at the end the following new item: "Section 45O. Contributions benefiting science, technology, engineering, and mathematics education at the elementary and secondary school level.". Effective Date. The amendments made by this section shall apply to taxable years beginning after the date of the enactment of this Act.'''
prompt = [
            {"role": "user",
             "content": "Given the following case, write a short summary of the case in 2-3 sentences:\n\nCase Summary: {}".format(text)},
    
        ]

prompt = pipe_finetuned.tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)

outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.1,
    top_k=20,
    top_p=0.3,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

Sure, here's a summary of the case in 2-3 sentences:

The case provides a tax incentive for businesses to contribute to STEM education programs at the elementary and secondary school level. The incentive is a 100 percent tax credit for STEM contributions, which include STEM school contributions, STEM teacher externship expenses, and STEM teacher training expenses.


In [23]:
import shutil
shutil.make_archive("results_1", 'zip', "/kaggle/working/final_model")

'/kaggle/working/results_1.zip'

In [39]:
from rouge import Rouge
def calculate_rouge_score(ref_summ, gen_summ):
    rouge = Rouge()

    # Calculate ROUGE scores
    scores = rouge.get_scores(gen_summ, ref_summ)

    # Extract ROUGE-1, ROUGE-2, and ROUGE-L scores
    rouge_1_score = scores[0]["rouge-1"]["f"]
    rouge_2_score = scores[0]["rouge-2"]["f"]
    rouge_l_score = scores[0]["rouge-l"]["f"]
    return rouge_1_score, rouge_2_score, rouge_l_score

case = '''SECTIONHEADER RELEASE OF DOCUMENTS CAPTURED IN IRAQ AND AFGHANISTAN. In General. The Director of National Intelligence shall make publicly available on an Internet website all captured documents. Review by Director of National Intelligence. The Director of National Intelligence may review a captured document before making such document publicly available under subsection (a). The Director shall not be required to make a captured document publicly available under subsection (a) if in the case of a captured document that is reviewed by the Director before the date of the enactment of this Act, the Director submits to the relevant congressional committees a description of the criteria the Director used to determine it is not appropriate to make a captured document publicly available and such captured document meets such criteria; or in the case of a captured document that is reviewed by the Director on or after the date of the enactment of this Act, the Director submits to the relevant congressional committees a description of the criteria the Director shall use to determine if it is not appropriate to make a captured document publicly available and the captured document meets such criteria. Submission of Description of Non-Released Documents. Review before date of enactment. Not later than 90 days after the date of the enactment of this Act, the Director of National Intelligence shall submit to the relevant congressional committees a report containing a description of each captured document that, before such date, the Director determined should not be made publicly available; and an explanation as to why the Director does not consider it appropriate to make such captured document publicly available. Review after date of enactment. Not later than 30 days after the Director of National Intelligence determines that a captured document should not be made publicly available pursuant to subsection (b)(2), the Director shall submit to the relevant congressional committees a report containing a description of such captured document and an explanation as to why the Director does not consider it appropriate to make such document publicly available. Request for document. The Director of National Intelligence shall make a copy of each captured document available to the relevant congressional committees for review upon request of the Chairman of any of such relevant congressional committees. The Director shall make such copy available in either classified or unclassified form. Publication or Review Date. In general. The Director of National Intelligence shall begin making captured documents publicly available pursuant to subsection (a) not later than 30 days after the date of the enactment of this Act. Documents collected prior to date of enactment. In general. Not later than the date described in subparagraph (B), for each captured document captured or collected before the date of the enactment of this Act, the Director of National Intelligence shall make such captured document publicly available pursuant to subsection (a) or shall submit to the relevant congressional committees a report regarding such captured document pursuant to subsection (c). Dates. The date described in this subparagraph is September 30, 2006, for captured documents captured or collected during Operation Enduring Freedom and Operation Iraqi Freedom; and March 31, 2007, for captured documents captured or collected during Operation Desert Storm. Documents collected after date of enactment. For each captured document that is captured or collected on or after the date of the enactment of this Act, not later than 60 days after the date on which such captured document is captured or collected, the Director of National Intelligence shall make such captured document publicly available pursuant to subsection (a) or shall submit to the relevant congressional committees a report regarding such captured document pursuant to subsection (c). Weekly Report. Not later than 7 days after the date of enactment of this Act, and weekly thereafter until each captured document captured or collected before the date of the enactment of this Act is made publicly available pursuant to subsection (a) or described in a report submitted pursuant to subsection (c), the Director of National Intelligence shall submit to the relevant congressional committees a report describing the progress in making captured documents publicly available. Definitions. In this section: Captured document. The term "captured document" means a document captured or collected in Afghanistan or Iraq, including a document collected from the Government of Iraq or from a private person and including a document in electronic form, during Operation Desert Storm, Operation Enduring Freedom, and Operation Iraqi Freedom. Relevant congressional committees. The term "relevant congressional committees" means the Permanent Select Committee on Intelligence of the House of Representatives and Select Committee on Intelligence of the Senate.'''
summ = '''Requires the Director of National Intelligence to make publicly available on an Internet website all documents captured in Afghanistan or Iraq during Operations Desert Storm, Enduring Freedom, or Iraqi Freedom. Provides conditions under which the Director shall not be required to make a captured document publicly available, including providing a list of retained documents, and the criteria used for such retention, to the congressional intelligence committees.'''
prompt = [
        {"role": "user",
         "content": "Given the following case, write a short summary of the case in 2-3 sentences:\n\nCase Summary: {}".format(case)},

    ]

prompt = pipe_finetuned.tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)

outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.1,
    top_k=20,
    top_p=0.3,
    add_special_tokens=True
)
generated_summ = outputs[0]["generated_text"][len(prompt):]
print("original_summ:\n")
print(summ)
print("generated text:\n")
print(generated_summ)
print("\n\n")
r1,r2,rl = calculate_rouge_score(generated_summ, summ)
print("r1 : "+str(r1))
print("r2 : "+str(r2))
print("rL : "+str(rl))

original_summ:

Requires the Director of National Intelligence to make publicly available on an Internet website all documents captured in Afghanistan or Iraq during Operations Desert Storm, Enduring Freedom, or Iraqi Freedom. Provides conditions under which the Director shall not be required to make a captured document publicly available, including providing a list of retained documents, and the criteria used for such retention, to the congressional intelligence committees.
generated text:

The case is about the release of captured documents from Iraq and Afghanistan. The Director of National Intelligence has made publicly available all captured documents, but some of the documents have been reviewed by the Director before being made publicly available. The Director has submitted a report to the relevant congressional committees describing the criteria that the Director used to determine which documents should be made publicly available.



r1 : 0.44444443948576673
r2 : 0.112903220807